In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline



In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [3]:
#train.head()
#Vanvittig mange variable, allerede før der laves dummies.
#PCA er formentlig en god idé her, efter alt feature engineering er færdig
#train.RoofStyle.value_counts()
#test.RoofStyle.value_counts()
#train.info()

In [4]:
#test["SalePrice"]
#test(['SalePrice'], axis=1, inplace=True)
#obj_df = df.select_dtypes(include=['object']).copy()
#object_clmns = train.dtypes[train.dtypes != "int64"].index
#print(object_clmns)
#print(train.columns)
print(pd.unique(train['MSZoning']))


['RL' 'RM' 'C (all)' 'FV' 'RH']


In [109]:
#for col in train.columns:
#    if train[col].dtype != "int64":
#        bla = pd.unique(train[col]) 
#        for blatest in bla:     
#            if len(train.loc[train[col] == blatest,col]) < 50:
#                train.loc[train[col] == blatest, col] = "Test"

In [110]:
#train1 = [train['MSZoning'] == 'RH']
#mask = df['position'] >= 20
#print(train1)


In [111]:
#sns.distplot(train['SalePrice'])
#Herre skæv fordeling

In [5]:
#Log transofmration af SalePrice log(1+x)
train["SalePrice"] = np.log1p(train["SalePrice"])
#sns.distplot(train['SalePrice'])
#Meget bedre, så kan linære modeller bruges bedre. 

In [6]:
#Samler train og test set, så vi ikke skal lave rettelser 2 gange. Gemmer 
trainrækker = train.shape[0]
testrækker = test.shape[0]

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

#Fjerner SalePrice så det ikke skal specificeres at den skal ekskluderes i alle operationer/transofmrationer af datasættet
y_train = train.SalePrice.values
#Resetter index så man kan kalde dem
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 79)


In [7]:
antal_NA= all_data.isnull().sum()
#Der er temmelig mange NA's
antal_NA.sort_values(ascending=False)[:20]

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
FireplaceQu     1420
LotFrontage      486
GarageFinish     159
GarageQual       159
GarageYrBlt      159
GarageCond       159
GarageType       157
BsmtCond          82
BsmtExposure      82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
BsmtHalfBath       2
dtype: int64

In [8]:
#PoolQC er poolquality
#Skyldes det måske at husene bare ikke har en pool og der derfor bliver rapporteret NA?
len(all_data)
#Det er 2909/2919 variable der er NAN
all_data['PoolQC'].describe() #Den er ordinal/kategorisk
pd.unique(all_data['PoolQC']) 
#Går ud fra at Ex er ekstremt godt, Fa er Fantastic og Gd bare er godt. Mærkeligt at der ikke er nogle i dårlig stand?

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)

In [9]:
all_data["PoolQC"].head()
all_data["PoolQC"] = all_data["PoolQC"].fillna("Unknown")
pd.unique(all_data['PoolQC']) 

array(['Unknown', 'Ex', 'Fa', 'Gd'], dtype=object)

In [10]:
pd.unique(all_data['MiscFeature']) #Nominel feature
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("Unknown")
pd.unique(all_data['MiscFeature']) 

array(['Unknown', 'Shed', 'Gar2', 'Othr', 'TenC'], dtype=object)

In [11]:
pd.unique(all_data['Alley']) 
all_data["Alley"] = all_data["Alley"].fillna("Unknown")
pd.unique(all_data['Alley']) 

array(['Unknown', 'Grvl', 'Pave'], dtype=object)

In [12]:
pd.unique(all_data['Fence']) 
all_data["Fence"] = all_data["Fence"].fillna("Unknown")
pd.unique(all_data['Fence']) 

array(['Unknown', 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'], dtype=object)

In [13]:
pd.unique(all_data['FireplaceQu'])
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("Unknown")
pd.unique(all_data['FireplaceQu']) 

array(['Unknown', 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [14]:
#pd.unique(all_data['LotFrontage'])#integers, enten med median eller mean?
#Grupperer dem efter neighborhood
list(all_data.groupby("Neighborhood")["LotFrontage"])
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [15]:
#Samler kategoriske/nominale features og erstatter i et loop istedet
#GarageQual GarageCond GarageFinish GarageType BsmtQual BsmtExposure BsmtCond BsmtFinType2 BsmtFinType1 MasVnrType MSZoning
pd.unique(all_data['MSSubClass']) 

array([ 60,  20,  70,  50, 190,  45,  90, 120,  30,  85,  80, 160,  75,
       180,  40, 150], dtype=int64)

In [16]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'SaleType',
            'BsmtExposure', 'BsmtCond','BsmtFinType2', 'BsmtFinType1', 'MasVnrType','MSZoning'):
    all_data[col] = all_data[col].fillna('Unknown')
pd.unique(all_data['Exterior2nd'])

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock', nan], dtype=object)

In [17]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtFinSF1',
           'BsmtFinSF1', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2'):
    all_data[col] = all_data[col].fillna(0)

In [18]:
all_data = all_data.drop(['Utilities'], axis=1)

In [19]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

In [20]:
antal_NA2= all_data.isnull().sum()
#Ingen NA's tilbage, yay!!!
antal_NA2.sort_values(ascending=False)[:5]

YrSold         0
Foundation     0
ExterCond      0
ExterQual      0
Exterior1st    0
dtype: int64

In [21]:
#for col in all_data.columns:
#    if all_data[col].dtype != "int64":
#        bla = pd.unique(all_data[col]) 
#        for blatest in bla:     
#            if len(all_data.loc[all_data[col] == blatest, col]) < 50:
#                all_data.loc[all_data[col] == blatest, col] = "Test"

In [22]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
#Looper over kolonner og konverterer kategoriske variable
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2919, 78)


In [23]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)
pd.unique(all_data['FireplaceQu'])

(2919, 223)


array([5, 4, 2, 1, 0, 3], dtype=int64)

In [24]:
train = all_data[:trainrækker]
test = all_data[trainrækker:]

In [25]:
train.head()
#test.shape
#test.columns
#Hvorfor helvede har test og train lige mange kolonner??
#For fanden, husk at fjerne SalePrice ordentligt!!!!

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_Unknown,SaleType_WD
0,856,854,0,2,3,3,3,706.0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
1,1262,0,0,2,3,3,1,978.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
2,920,866,0,2,3,3,2,486.0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
3,961,756,0,2,3,1,3,216.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
4,1145,1053,0,2,4,3,0,655.0,0.0,2,...,0,0,0,0,0,0,0,0,0,1


In [26]:
#Modeller der evt. kan testes:
#OLS, Lasso, Ridge, Gradient Boosting, RandomForest, LightGBM, XGBoost, KNN med forskellige antal k'er?, 
#Måske kunne det være ret sejt at stacke det ind i et NN

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [27]:
#Evalueringen er rmsle, så eval metric'en der skal bruges skal være rmse
#Måske er 5 folds alt for meget? Virkelig et lille datasæt trods alt
n_folds = 5
#Bruger 5kfolds til at træne modellen og give rmse tilbage
def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(kfoldtrain.values)
    rmse= np.sqrt(-cross_val_score(model, kfoldtrain.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

kfoldtrain = train
#Skal først bruges når der skal stackes, parameterne skal findes først. Evt. genbrug randomsearch du lavede til taxa challenge

In [28]:
train["SalePrice"] = y_train
#Train, Test = train_test_split(train, test_size = 0.3)
#X_train = Train.drop(['SalePrice'], axis=1)
#Y_train = Train["SalePrice"]
#X_test = Test.drop(['SalePrice'], axis=1)
#Y_test = Test["SalePrice"]

#Y_test = Y_test.reset_index().drop('index',axis = 1)
#Y_train = Y_train.reset_index().drop('index',axis = 1)

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso
print("\nLasso score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))

NameError: name 'score' is not defined

In [31]:
n_folds = 5

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state = 0).get_n_splits(train)
    rmse = np.sqrt(-cross_val_score(model, train, y_train, scoring ="neg_mean_squared_error", cv=kf))
    return(rmse)


In [32]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=0))

In [33]:
score = rmse_cv(lasso)
print("\nLasso score: {:.4f} (+/-{:.4f})\n" .format(score.mean(), score.std()))



Lasso score: 0.0007 (+/-0.0000)



In [34]:
lasso2 = LassoCV()

NameError: name 'LassoCV' is not defined

In [35]:
score.mean()

0.0006682663752289137

In [36]:
class SklearnHelper(object):
    def __init__(self, regr, seed = 0, params = None):
        #Indsætter seed, så alle styres med samme seed og er dermed sammenlignlige
        params["random_state"] = seed
        #Indsætter parametrene til den enkelte model
        self.regr = regr(**params)
        
        #Herefter sætter vi den til at træne
    def train(self, x_train, y_train):
        self.regr.fit(x_train, y_train)
        
        #Herefter predictes x?
    def predict(self, x):
        return self.regr.predict(x)
        
        
    def fit(self, x, y):
        return self.regr.fit(x,y)
    
    def feature_importances(self, x, y):
        print(self.regr.fit(x, y).feature_importances_)
    

    
        

In [37]:
#### Iterationer
#Træningssæt, validation sæt og test sæt - Lightgbm skal også have deres egne. Alternativt kan vi lade være med at 
#bruge deres specialdesignet matricer men tror ikke det kan undgås til selve tuningen, når at watchlist skal laves?
dtrain = xgb.DMatrix(X_train, label=Y_train)
dvalid = xgb.DMatrix(X_test, label=Y_test)
#dtest = xgb.DMatrix(test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]


#VI SKAL EKSPERIMENTERE MED BOOST TYPE OGSÅ!!!!!
#gboost has access to three different boosting methods:
#- "gblinear" (Generalized Linear Model) which is using Shotgun (Parallel Stochastic Gradient Descent);
#- "gbtree" (Gradient Boosted Trees) which is the default boosting method using Decision Trees and Stochastic Gradient Descent;
#- "dart" (Dropout Additive Regression Trees) which is a method employing the Dropout method from Neural Networks.
#Defaults to "gbtree".
Iter = 100
best_score_true = 1

xgb_pars = []
for MCW in [21]:
    #for ETA in [0.05, 0.1, 0.15]:
        for CS in [0.5]:
            for MD in [13]:
                for SS in [0.85]:
                    for LAMBDA in [1.7]:
                        xgb_pars.append({'min_child_weight': MCW, 'eta': 0.01, 
                                         'colsample_bytree':CS, 'max_depth': MD,
                                         'subsample': SS, 'lambda': LAMBDA, 
                                         'nthread': -1, 'booster' : 'gbtree', 'eval_metric': 'rmse',
                                         'silent': 1, 'objective': 'reg:linear', 'seed':42})
#i = 0
print(len(xgb_pars))
for i in range(len(xgb_pars)):
#while i < Iter:
   # xgb_par = np.random.choice(xgb_pars, replace = False)
    xgb_par = xgb_pars[i]
    print(xgb_par)
    model = xgb.train(xgb_par, dtrain, 3000, watchlist, early_stopping_rounds=150,
                      maximize=False, verbose_eval=100)
   # i+= 1
    if model.best_score < best_score_true:
        best_score_true = model.best_score
        xgb_par_best = xgb_par

    print('Modeling RMSLE %.5f\n' % model.best_score)

NameError: name 'X_train' is not defined

In [35]:
print('Modeling RMSLE %.5f\n' % best_score_true, xgb_par_best)

#Modeling RMSLE 0.11743
# {'min_child_weight': 20, 'eta': 0.2, 'colsample_bytree': 0.5, 'max_depth': 12, 
#'subsample': 0.9, 'lambda': 2, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse',
#'silent': 1, 'objective': 'reg:linear', 'seed': 42}

#Modeling RMSLE 0.09561
# {'min_child_weight': 22, 'eta': 0.2, 'colsample_bytree': 0.5, 'max_depth': 12, 
#  'subsample': 0.85, 'lambda': 1.8, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse',
#  'silent': 1, 'objective': 'reg:linear', 'seed': 42}

#Ser godt ud til vi kan teste højere lambda og max depth faktisk!

Modeling RMSLE 0.11812
 {'min_child_weight': 21, 'eta': 0.01, 'colsample_bytree': 0.5, 'max_depth': 13, 'subsample': 0.85, 'lambda': 1.7, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear', 'seed': 42}


In [36]:
testmatrix = xgb.DMatrix(test)

In [38]:
train["SalePrice"] = y_train
#x2_train['SalePrice']=y_train

In [39]:
n_iters = 5
xgb_preds = []
for i in range(n_iters): 
    Train, Test = train_test_split(train, test_size = 0.2, random_state = i)
    X_train = Train.drop(['SalePrice'], axis=1)
    Y_train = Train["SalePrice"]
    X_test = Test.drop(['SalePrice'], axis=1)
    Y_test = Test["SalePrice"]

    Y_test = Y_test.reset_index().drop('index',axis = 1)
    Y_train = Y_train.reset_index().drop('index',axis = 1)
    
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dvalid = xgb.DMatrix(X_test, label=Y_test)
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    xgb_model = xgb.train(xgb_par_best, dtrain, 20000, watchlist, early_stopping_rounds=150,
                      maximize=False, verbose_eval=500)
    preds = xgb_model.predict(testmatrix)
    preds = np.exp(preds) - 1
    xgb_preds.append(preds)

NameError: name 'xgb_par_best' is not defined

In [40]:
np.mean(xgb_preds, axis=0)

nan

In [41]:
predsfinal =[]

In [42]:
predsfinal = np.mean(xgb_preds, axis=0)

In [43]:
predsfinal

nan

In [44]:
predictions = pd.DataFrame(list(zip(predsfinal)),
              columns=['xgbpreds'])

TypeError: zip argument #1 must support iteration

In [45]:
predictions.head()

NameError: name 'predictions' is not defined

In [46]:
#### Iterationer
#Træningssæt, validation sæt og test sæt - Lightgbm skal også have deres egne. Alternativt kan vi lade være med at 
#bruge deres specialdesignet matricer men tror ikke det kan undgås til selve tuningen, når at watchlist skal laves?
X2_train = np.array(X_train)
Y2_train = np.array(Y_train)
X2_test  = np.array(X_test)
Y2_test  = np.array(Y_test)
#Skal gøres 1 dimensionel for at lgb spiser dem
y=Y2_train.ravel()
y2 = Y2_test.ravel()
d_train = lgb.Dataset(X2_train, label=y)
d_valid = lgb.Dataset(X2_test, label=y2)
watchlist = [d_valid]

#VI SKAL EKSPERIMENTERE MED BOOST TYPE OGSÅ!!!!!
#gboost has access to three different boosting methods:
#- "gblinear" (Generalized Linear Model) which is using Shotgun (Parallel Stochastic Gradient Descent);
#- "gbtree" (Gradient Boosted Trees) which is the default boosting method using Decision Trees and Stochastic Gradient Descent;
#- "dart" (Dropout Additive Regression Trees) which is a method employing the Dropout method from Neural Networks.
#Defaults to "gbtree".
#Iter = 100
bestlgb_score_true = 1
n_estimators = 1000
#'early_stopping': 10

lgb_pars = []
for MCW in [20]:
    #for ETA in [0.05, 0.1, 0.15]:
        for CS in [0.85]:
            for MD in [16]:
                for SS in [0.85]:
                    for LAMBDA in [2.1]:
                        lgb_pars.append({'learning_rate': 0.01, 'min_sum_hessian_in_leaf': MCW,
                                         'feature_fraction':CS, 'max_depth': MD,
                                         'bagging_fraction': SS, 'lambda_l2': LAMBDA, 
                                         'boosting_type' : 'gbdt', 'metric': 'rmse',
                                          'objective': 'regression', 
                                           'verbose': 0})
#i = 0
print(len(lgb_pars))
for i in range(len(lgb_pars)):
#while i < Iter:
   # xgb_par = np.random.choice(xgb_pars, replace = False)
    lgb_par = lgb_pars[i]
    print(lgb_par)
    lgb_model = lgb.train(lgb_par, d_train, n_estimators, watchlist, verbose_eval = 500, early_stopping_rounds = 50)
    if int(lgb_model.best_score['valid_0']['rmse']) < bestlgb_score_true:
        bestlgb_score_true = lgb_model.best_score['valid_0']['rmse']
        lgb_par_best = lgb_par
        
    #print('Modeling RMSLE %.5f\n' % bestlgb_score_true, lgb_par_best)

1
{'learning_rate': 0.01, 'min_sum_hessian_in_leaf': 20, 'feature_fraction': 0.85, 'max_depth': 16, 'bagging_fraction': 0.85, 'lambda_l2': 2.1, 'boosting_type': 'gbdt', 'metric': 'rmse', 'objective': 'regression', 'verbose': 0}
Training until validation scores don't improve for 50 rounds.
[500]	valid_0's rmse: 0.134097
[1000]	valid_0's rmse: 0.131766
Did not meet early stopping. Best iteration is:
[987]	valid_0's rmse: 0.131721


In [47]:
print(lgb_par)

{'learning_rate': 0.01, 'min_sum_hessian_in_leaf': 20, 'feature_fraction': 0.85, 'max_depth': 16, 'bagging_fraction': 0.85, 'lambda_l2': 2.1, 'boosting_type': 'gbdt', 'metric': 'rmse', 'objective': 'regression', 'verbose': 0, 'max_bin': 255}


In [48]:
print('Modeling RMSLE %.5f\n' % bestlgb_score_true, lgb_par_best)

Modeling RMSLE 0.11135
 {'learning_rate': 0.01, 'min_sum_hessian_in_leaf': 20, 'feature_fraction': 0.85, 'max_depth': 16, 'bagging_fraction': 0.85, 'lambda_l2': 2.1, 'boosting_type': 'gbdt', 'metric': 'rmse', 'objective': 'regression', 'verbose': 0, 'max_bin': 255}


In [49]:
lgb_pars_final = []
lgb_pars_final.append({'learning_rate': 0.01, 'min_sum_hessian_in_leaf': 20,
                                         'feature_fraction':0.85, 'max_depth': 16,
                                         'bagging_fraction': 0.85, 'lambda_l2': 2.1, 
                                         'boosting_type' : 'gbdt', 'metric': 'rmse',
                                          'objective': 'regression', 'seed':42, 
                                           'verbose': 0})
print(lgb_pars_final)

[{'learning_rate': 0.01, 'min_sum_hessian_in_leaf': 20, 'feature_fraction': 0.85, 'max_depth': 16, 'bagging_fraction': 0.85, 'lambda_l2': 2.1, 'boosting_type': 'gbdt', 'metric': 'rmse', 'objective': 'regression', 'seed': 42, 'verbose': 0}]


In [50]:
n_iters = 5
n_estimators = 20000
lgb_preds = []
for i in range(n_iters): 
    Train, Test = train_test_split(train, test_size = 0.2, random_state = i)
    X_train = Train.drop(['SalePrice'], axis=1)
    Y_train = Train["SalePrice"]
    X_test = Test.drop(['SalePrice'], axis=1)
    Y_test = Test["SalePrice"]

    Y_test = Y_test.reset_index().drop('index',axis = 1)
    Y_train = Y_train.reset_index().drop('index',axis = 1)
    
    X2_train = np.array(X_train)
    Y2_train = np.array(Y_train)
    X2_test  = np.array(X_test)
    Y2_test  = np.array(Y_test)
    #Skal gøres 1 dimensionel for at lgb spiser dem
    y=Y2_train.ravel()
    y2 = Y2_test.ravel()
    d_train = lgb.Dataset(X2_train, label=y)
    d_valid = lgb.Dataset(X2_test, label=y2)
    watchlist = [d_valid]

    lgb_model = lgb.train(lgb_par_best , d_train, n_estimators, watchlist, verbose_eval = 500, early_stopping_rounds = 150)

    preds = lgb_model.predict(test)
    preds = np.exp(preds) - 1
    lgb_preds.append(preds)

Training until validation scores don't improve for 150 rounds.
[500]	valid_0's rmse: 0.134073
[1000]	valid_0's rmse: 0.131886
[1500]	valid_0's rmse: 0.13061
[2000]	valid_0's rmse: 0.129905
[2500]	valid_0's rmse: 0.129197
[3000]	valid_0's rmse: 0.128607
[3500]	valid_0's rmse: 0.128122
[4000]	valid_0's rmse: 0.127782
[4500]	valid_0's rmse: 0.127453
[5000]	valid_0's rmse: 0.127158
[5500]	valid_0's rmse: 0.126887
[6000]	valid_0's rmse: 0.126654
[6500]	valid_0's rmse: 0.126438
[7000]	valid_0's rmse: 0.126298
[7500]	valid_0's rmse: 0.126174
[8000]	valid_0's rmse: 0.126083
[8500]	valid_0's rmse: 0.126001
[9000]	valid_0's rmse: 0.125917
[9500]	valid_0's rmse: 0.125857
[10000]	valid_0's rmse: 0.125801
[10500]	valid_0's rmse: 0.12575
[11000]	valid_0's rmse: 0.125713
[11500]	valid_0's rmse: 0.125677
[12000]	valid_0's rmse: 0.125648
[12500]	valid_0's rmse: 0.125624
[13000]	valid_0's rmse: 0.125603
[13500]	valid_0's rmse: 0.12558
[14000]	valid_0's rmse: 0.125549
[14500]	valid_0's rmse: 0.125525
[15

In [51]:
np.mean(lgb_preds[0])

176916.61507518287

In [52]:
lgb_predsmean = np.mean(lgb_preds, axis=0)

In [53]:
lgb_predsmean

array([ 120930.22710408,  155364.04502872,  186760.01936407, ...,
        158348.32028924,  116176.3662145 ,  222746.97438011])

In [54]:
predictions['lgbpreds'] = lgb_predsmean

In [55]:
predictions.head()

,xgbpreds,lgbpreds
0,122497.101562,120930.227104
1,159787.437500,155364.045029
2,187248.406250,186760.019364
3,191083.578125,186827.802887
4,185884.828125,185071.816570


In [56]:
from sklearn.linear_model import LassoCV, LassoLarsCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV

C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [57]:
train.drop(['SalePrice'], axis=1, inplace=True)

In [229]:
model_lasso = LassoCV(cv = kf, alphas = [1, 0.1, 0.001, 0.0005], selection='random', max_iter=20000).fit(train, y_train)

In [230]:
coef = pd.Series(model_lasso.coef_, index = train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

Lasso picked 100 variables and eliminated the other 123 variables


In [231]:
lasso_preds = np.expm1(model_lasso.predict(test))

In [61]:
from sklearn.feature_selection import SelectFromModel
selection = SelectFromModel(model_lasso, prefit=True)
select_X_train = selection.transform(train)

In [232]:
lasso_preds

array([ 117725.22065635,  146567.51658564,  173294.18689029, ...,
        164293.73482401,  113845.59607943,  229068.6831272 ])

In [63]:
x2_train = pd.DataFrame(select_X_train)

In [64]:
x2_train.head()

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,856.0,854.0,0.0,2.0,3.0,3.0,3.0,706.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1262.0,0.0,0.0,2.0,3.0,3.0,1.0,978.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,920.0,866.0,0.0,2.0,3.0,3.0,2.0,486.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,961.0,756.0,0.0,2.0,3.0,1.0,3.0,216.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,1145.0,1053.0,0.0,2.0,4.0,3.0,0.0,655.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [233]:
predictions['lassopreds'] = lasso_preds

In [234]:
predictions.head()

,xgbpreds,lgbpreds,lassopreds,elasticpreds,knnpreds
0,122497.101562,120930.227104,117725.220656,117627.559034,125137.178776
1,159787.437500,155364.045029,146567.516586,147009.326414,175728.889647
2,187248.406250,186760.019364,173294.186890,173220.654261,172717.511392
3,191083.578125,186827.802887,197096.093798,196875.689866,207897.454526
4,185884.828125,185071.816570,199137.426424,199966.054071,165335.487844


In [67]:
from xgboost import XGBRegressor

In [72]:
param = {
 'n_estimators':[500,800,1000,2000],
 'max_depth':[3,4,6,8,10],
 'min_child_weight':[4,6,8,10,12],
 'colsample_bytree':[0.2,0.6,0.8],
 'colsample_bylevel':[0.2,0.6,0.8]
}

In [73]:
gsearch1 = GridSearchCV(estimator = XGBRegressor( 
        objective= 'reg:linear', 
        seed=1), 
    param_grid = param, 
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose = 1)


In [74]:
#Den her tager 1000 år at blive færdig 
gsearch1.fit(train, y_train)

Fitting 5 folds for each of 900 candidates, totalling 4500 fits


KeyboardInterrupt: 

In [347]:
{'min_child_weight': 21, 'eta': 0.01, 'colsample_bytree': 0.5, 
 'max_depth': 13, 'subsample': 0.85, 'lambda': 1.7, 'nthread': -1, 'booster': 'gbtree', 
 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear', 'seed': 42}

{'booster': 'gbtree',
 'colsample_bytree': 0.5,
 'eta': 0.01,
 'eval_metric': 'rmse',
 'lambda': 1.7,
 'max_depth': 13,
 'min_child_weight': 21,
 'nthread': -1,
 'objective': 'reg:linear',
 'seed': 42,
 'silent': 1,
 'subsample': 0.85}

In [68]:
from sklearn.linear_model import ElasticNet

In [69]:
elastic = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)

In [70]:
elastic_model = elastic.fit(train, y_train)

In [71]:
elastic_preds = np.expm1(elastic_model.predict(test))

In [72]:
predictions['elasticpreds'] = elastic_preds

In [73]:
predictions.head()

,xgbpreds,lgbpreds,lassopreds,elasticpreds
0,122497.101562,120930.227104,117724.375824,117627.559034
1,159787.437500,155364.045029,146569.046524,147009.326414
2,187248.406250,186760.019364,173296.273512,173220.654261
3,191083.578125,186827.802887,197097.025555,196875.689866
4,185884.828125,185071.816570,199136.540047,199966.054071


In [74]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor()


In [75]:
knn_params = {"n_neighbors": np.arange(1, 31, 2), 'leaf_size': np.arange(1, 31, 2),
   "metric": ["euclidean", "cityblock"]}

In [76]:
grid = GridSearchCV(neigh, knn_params)

grid.fit(train, y_train)
grid.best_params_

{'leaf_size': 3, 'metric': 'cityblock', 'n_neighbors': 11}

In [77]:
knn_model = KNeighborsRegressor(n_neighbors = 11, leaf_size = 3, metric = 'cityblock')

In [78]:
knn_model.fit(train, y_train)


KNeighborsRegressor(algorithm='auto', leaf_size=3, metric='cityblock',
          metric_params=None, n_jobs=1, n_neighbors=11, p=2,
          weights='uniform')

In [79]:
knn_preds = np.expm1(knn_model.predict(test))
knn_preds

array([ 125137.17877574,  175728.88964683,  172717.51139237, ...,
        180693.5453129 ,  112099.94874035,  233997.44619259])

In [85]:
predictions['knnpreds'] = knn_preds

In [88]:
np.expm1(knn_model.predict(train))

array([ 208297.96157517,  169772.1379005 ,  207533.01212296, ...,
        211812.89357806,  133514.72260304,  158453.51795065])

In [84]:
predictions.head()

,xgbpreds,lgbpreds,lassopreds,elasticpreds,knnpreds
0,122497.101562,120930.227104,117724.375824,117627.559034,125137.178776
1,159787.437500,155364.045029,146569.046524,147009.326414,175728.889647
2,187248.406250,186760.019364,173296.273512,173220.654261,172717.511392
3,191083.578125,186827.802887,197097.025555,196875.689866,207897.454526
4,185884.828125,185071.816570,199136.540047,199966.054071,165335.487844


In [82]:
finalpredictions = predictions.mean(axis=1)

In [83]:
testpred = predictions['xgbpreds']*0.18 + predictions['lgbpreds']*0.21 + predictions['lassopreds']*0.3 + predictions['knnpreds']*0.00 + predictions['elasticpreds'] * 0.3

In [156]:
35/2

17.5

In [346]:
submission2 = pd.DataFrame()
submission2['Id'] = test_ID
submission2["SalePrice"] = testpred
submission2.head()
submission2.to_csv("submission.csv", index=False)